# Step 1: List all pools

In [2]:
# 📊 Data Handling
import pandas as pd
import numpy as np
import requests

# 📈 Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# ⚙️ Preprocessing & Scaling
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# 📈 Analysis
import scipy.stats as stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
  
# 🧠 Modeling
from sklearn.cluster import KMeans, DBSCAN
import hdbscan
from sklearn.metrics import silhouette_score

In [9]:
df = pd.read_csv('all_pools.csv',on_bad_lines='skip').sort_values(by='COUNT(TX_HASH)', ascending=False)

In [10]:
df.head(1)

,POOL_NAME,PLATFORM,COUNT(TX_HASH)
19361,WETH-USDT 100 1 UNI-V3 LP,uniswap-v3,1448252.0


In [11]:
df['PLATFORM'].value_counts()

uniswap-v2        45398
uniswap-v3         7748
uniswap-v4         3908
sushiswap          1874
dodo-v2             782
balancer            476
pancakeswap-v2      438
shibaswap           418
curve               357
hashflow-v3         353
pancakeswap-v3      260
woofi               137
kyberswap-v1         27
verse                22
kyberswap-v2         17
fraxswap             11
dodo-v1              10
trader-joe-v2         1
maverick              1
Name: PLATFORM, dtype: int64

In [12]:
df.loc[df["POOL_NAME"].str.contains("WETH-USDT", case=False, na=False), "pool_type"] = "Major Token Pair"
df.loc[df["POOL_NAME"].str.contains("USDC-WETH", case=False, na=False), "pool_type"] = "Major Token Pair"
df.loc[df["POOL_NAME"].str.contains("ETH-USDC", case=False, na=False), "pool_type"] = "Major Token Pair"
df.loc[df["POOL_NAME"].str.contains("WBTC-WETH", case=False, na=False), "pool_type"] = "Major Token Pair"
df.loc[df["POOL_NAME"].str.contains("WBTC-USDC", case=False, na=False), "pool_type"] = "Major Token Pair"
df.loc[df["POOL_NAME"].str.contains("WBTC-USDT", case=False, na=False), "pool_type"] = "Major Token Pair"
df.loc[df["POOL_NAME"].str.contains("WBTC-USDC", case=False, na=False), "pool_type"] = "Major Token Pair"
df.loc[df["POOL_NAME"].str.contains("tBTC-WBTC", case=False, na=False), "pool_type"] = "Major Token Pair"
#df.loc[df["POOL_NAME"].str.contains("WBTC-USDC", case=False, na=False), "pool_type"] = "Major Token Pair"
#df.loc[df["POOL_NAME"].str.contains("WBTC-USDC", case=False, na=False), "pool_type"] = "Major Token Pair"
#df.loc[df["POOL_NAME"].str.contains("WBTC-USDC", case=False, na=False), "pool_type"] = "Major Token Pair"





df.loc[df["POOL_NAME"].str.contains("USDC-USDT", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("crvUSD/USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("crvUSD/USDT", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("USDe-USDT", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("DAI-USDT", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("DAI-USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("USDe-USDC", case=False, na=False), "pool_type"] = "Stable Pair"




df.loc[df["POOL_NAME"].str.contains("KEKIUS", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("WHITE-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("9279", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("WBTC-cbBTC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDC-SEI", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("PLUME-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("DOLO-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("PROMPT-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("VATAN-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("UNI-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("CGPT-USDT", case=False, na=False), "pool_type"] = "Other"




df.loc[df["POOL_NAME"].str.contains("LINK-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("SWFTC-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("WBTC-BADGER", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USUALX-USUAL", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("ENA-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("RAD-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("PAXG-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDT-MOCA", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("UNI-AAVE", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("ZND-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("OMIKAMI-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDC-STG", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("deUSD-USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("USR-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("WBTC-LBTC", case=False, na=False), "pool_type"] = "Major Token Pair"


df.loc[df["POOL_NAME"].str.contains("FRAX-USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("AAVE-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("cvxCrv/Crv", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USD0-USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("SHFL-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("EUROC-USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("FXS-FRAX", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USD0-USDT", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("OBOL-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("DRAGONX-TITANX", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("VEE-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("MMX-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("AXL-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("KERNEL-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("TKX-USDT", case=False, na=False), "pool_type"] = "Other"

In [13]:
df.loc[df["POOL_NAME"].str.contains("SD-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("CPOOL-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDC-MYTH", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("G-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDT-USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("DOGE-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("CAH-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("NEAR-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("ENS-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("ALI-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("TARA-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("VOLT-TITANX", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("msUSD/FRAXBP", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("0x315...784f-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("MASK-USDC", case=False, na=False), "pool_type"] = "Other"

In [14]:
df.loc[df["POOL_NAME"].str.contains("GHO-USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("LINK-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("RLB-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("ENA-sENA", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("HEX-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("GASP-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("RLB-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("TRX-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("PROPC-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("TEL-GMRT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USD1-USDC", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("USDT-USDf", case=False, na=False), "pool_type"] = "Stable Pair"
df.loc[df["POOL_NAME"].str.contains("BITCOIN-SPX", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("33APU-33PEPE-33SPX", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDC-cbBTC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("NUMI-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("INV-DOLA-DBR", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("wA7A5-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("PEAS-DAI", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDT-TRU", case=False, na=False), "pool_type"] = "Other"

In [15]:
df.loc[df["POOL_NAME"].str.contains("ELX-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("XAUt-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDC-CRV", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("BKN-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("DHN-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("ID-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("PTC-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("cvxCRV-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("USDC-SHELL", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("BLZ-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("UNI-WBTC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("ASTO-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("AAVE-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("50COW-50GNO", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("OZK-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("OX-SPX", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("WAXP-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("VR-USDT", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("ID-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("IQ-FRAX", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("REZ-USDC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("WBTC-SolvBTC", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("X28-TITANX", case=False, na=False), "pool_type"] = "Other"
df.loc[df["POOL_NAME"].str.contains("WHITE-USDT", case=False, na=False), "pool_type"] = "Other"

In [16]:
df[df['pool_type'].isnull()].head(3)

,POOL_NAME,PLATFORM,COUNT(TX_HASH),pool_type
25700,ETH-USDT 500 10 UNI-V4 LP,uniswap-v4,213905.0,NaN
27459,DAI-WETH 500 10 UNI-V3 LP,uniswap-v3,202293.0,NaN
39877,WETH-SPX,uniswap-v2,178729.0,NaN


In [17]:
df['pool_type'].value_counts()

Other               488
Stable Pair         216
Major Token Pair    151
Name: pool_type, dtype: int64

# Step 2: Filter all pools belong to curve

In [65]:
df[(df['PLATFORM']=="curve") & (df['pool_type']=="Stable Pair")]['POOL_NAME'].to_list()

['DAI-USDC-USDT',
 'Curve.fi Factory Plain Pool: crvUSD/USDC',
 'Curve.fi Factory Plain Pool: crvUSD/USDT',
 'Curve.fi Pool: DAI-USDC-USDT-sUSD',
 'FRAX-USDC',
 'Curve.fi Factory USD Metapool: msUSD/FRAXBP',
 'RAI-DAI-3Crv-USDC-USDT',
 'aDAI-aUSDT-DAI-USDC-aUSDC-USDT',
 'GUSD-DAI-3Crv-USDC-USDT',
 'Curve.fi Pool: cDAI-cUSDC-USDT',
 'cyUSDT-DAI-cyUSDC-cyDAI-USDC-USDT',
 'USDC-USDT',
 'DAI-3Crv-USDC-USDT-mUSD',
 'USDK-DAI-3Crv-USDC-USDT',
 'USDN-DAI-3Crv-USDC-USDT',
 'RSV-DAI-3Crv-USDC-USDT',
 'USDP-DAI-3Crv-USDC-USDT',
 'DUSD-DAI-3Crv-USDC-USDT',
 'LINKUSD-DAI-3Crv-USDC-USDT',
 'DAI-3Crv-USDC-USDT-HUSD']

In [2]:
# query for SWAP_COUNT, TOTAL_VOLUME_USD, MIN_DEPTH_USD, AVG_SLIPPAGE, MAX_SLIPPAGE

from flipside import Flipside

flipside = Flipside("bfb1c2a7-30de-46af-a084-5b8d9511280c", "https://api-v2.flipsidecrypto.xyz")

sql = """
SELECT
s.BLOCK_TIMESTAMP as date,
DATE_TRUNC('hour', s.BLOCK_TIMESTAMP) AS hour,
s.POOL_NAME as pool, 
count(TX_HASH) as SWAP_COUNT,
sum(s.AMOUNT_IN_USD + s.AMOUNT_OUT_USD) as TOTAL_VOLUME_USD,
min(s.AMOUNT_IN_USD + s.AMOUNT_OUT_USD) as MIN_DEPTH_USD,
round(AVG((ABS(s.AMOUNT_IN_USD - s.AMOUNT_OUT_USD) / s.AMOUNT_IN_USD) * 100),4) as AVG_SLIPPAGE,
round(MAX( ABS(s.AMOUNT_IN_USD - s.AMOUNT_OUT_USD) / NULLIF(s.AMOUNT_IN_USD, 0) * 100),4) as MAX_SLIPPAGE,
max(s.AMOUNT_OUT_USD / NULLIF(s.AMOUNT_IN_USD, 0)) AS max_price,
min(s.AMOUNT_OUT_USD / NULLIF(s.AMOUNT_IN_USD, 0)) AS min_price,
avg(s.AMOUNT_OUT_USD / NULLIF(s.AMOUNT_IN_USD, 0)) AS avg_price
from ethereum.defi.ez_dex_swaps s
where s.BLOCK_TIMESTAMP between '2025-04-01' AND TIMESTAMP '2025-04-07'
 and s.AMOUNT_IN_USD > 100
and LOWER(s.PLATFORM) = 'curve'
and  s.POOL_NAME IN (
  'DAI-USDC-USDT',
  'Curve.fi Factory Plain Pool: crvUSD/USDC',
  'Curve.fi Factory Plain Pool: crvUSD/USDT',
  'Curve.fi Pool: DAI-USDC-USDT-sUSD',
  'FRAX-USDC',
  'Curve.fi Factory USD Metapool: msUSD/FRAXBP',
  'RAI-DAI-3Crv-USDC-USDT',
  'aDAI-aUSDT-DAI-USDC-aUSDC-USDT',
  'GUSD-DAI-3Crv-USDC-USDT',
  'Curve.fi Pool: cDAI-cUSDC-USDT',
  'cyUSDT-DAI-cyUSDC-cyDAI-USDC-USDT',
  'USDC-USDT',
  'DAI-3Crv-USDC-USDT-mUSD',
  'USDK-DAI-3Crv-USDC-USDT',
  'USDN-DAI-3Crv-USDC-USDT',
  'RSV-DAI-3Crv-USDC-USDT',
  'USDP-DAI-3Crv-USDC-USDT',
  'DUSD-DAI-3Crv-USDC-USDT',
  'LINKUSD-DAI-3Crv-USDC-USDT',
  'DAI-3Crv-USDC-USDT-HUSD'
)
group by 1,2,3


"""

query_result_set = flipside.query(sql)
df_c = pd.DataFrame(query_result_set.records)

In [ ]:
# query for AVG_TVL_USD

flipside = Flipside("bfb1c2a7-30de-46af-a084-5b8d9511280c", "https://api-v2.flipsidecrypto.xyz")

sql = """
WITH swap_token AS 
(SELECT
s.BLOCK_TIMESTAMP as date,
DATE_TRUNC('hour', s.BLOCK_TIMESTAMP) AS hour,
s.POOL_NAME as pool, 
SPLIT_PART(s.POOL_NAME, '-', 1) AS SYMBOL_IN,
s.AMOUNT_IN,
s.AMOUNT_OUT,
REVERSE(SPLIT_PART(REVERSE(s.POOL_NAME), '-', 1)) AS SYMBOL_OUT
from ethereum.defi.ez_dex_swaps s
where s.BLOCK_TIMESTAMP between '2025-04-01' AND TIMESTAMP '2025-04-07'
 and s.AMOUNT_IN_USD > 100
and LOWER(s.PLATFORM) = 'curve'
and  s.POOL_NAME IN (
  'DAI-USDC-USDT',
  'Curve.fi Factory Plain Pool: crvUSD/USDC',
  'Curve.fi Factory Plain Pool: crvUSD/USDT',
  'Curve.fi Pool: DAI-USDC-USDT-sUSD',
  'FRAX-USDC',
  'Curve.fi Factory USD Metapool: msUSD/FRAXBP',
  'RAI-DAI-3Crv-USDC-USDT',
  'aDAI-aUSDT-DAI-USDC-aUSDC-USDT',
  'GUSD-DAI-3Crv-USDC-USDT',
  'Curve.fi Pool: cDAI-cUSDC-USDT',
  'cyUSDT-DAI-cyUSDC-cyDAI-USDC-USDT',
  'USDC-USDT',
  'DAI-3Crv-USDC-USDT-mUSD',
  'USDK-DAI-3Crv-USDC-USDT',
  'USDN-DAI-3Crv-USDC-USDT',
  'RSV-DAI-3Crv-USDC-USDT',
  'USDP-DAI-3Crv-USDC-USDT',
  'DUSD-DAI-3Crv-USDC-USDT',
  'LINKUSD-DAI-3Crv-USDC-USDT',
  'DAI-3Crv-USDC-USDT-HUSD'
))
select
date, 
t.hour,
POOL,
sum((t.AMOUNT_IN * p0.PRICE) + (t.AMOUNT_OUT * p1.PRICE)) as tvl
from swap_token t 
left join ethereum.price.ez_prices_hourly p0 on t.SYMBOL_IN = p0.SYMBOL 
and t.hour = p0.HOUR
left join ethereum.price.ez_prices_hourly p1 on t.SYMBOL_OUT = p1.SYMBOL 
and t.hour = p1.HOUR
group by 1,2,3



"""

query_result_set = flipside.query(sql)
df_c_tvl = pd.DataFrame(query_result_set.records)


In [66]:
# temporary import data
df_c = pd.read_csv('01_df_metrics.csv',on_bad_lines='skip')
df_c_tvl = pd.read_csv('01_df_tvl.csv',on_bad_lines='skip')

In [68]:
# extract date & hour
df_c['DATE'] = df_c['DATE'].astype(str).str.replace('\ufeff', '').str.strip()
df_c['HOUR'] = df_c['HOUR'].astype(str).str.replace('\ufeff', '').str.strip()

df_c['DATE'] = pd.to_datetime(df_c['DATE'], errors='coerce')
df_c['HOUR'] = pd.to_datetime(df_c['HOUR'], errors='coerce')

df_c['DATE_ONLY'] = df_c['DATE'].dt.date
df_c['HOUR_ONLY'] = df_c['HOUR'].dt.hour

cols = ['DATE_ONLY', 'HOUR_ONLY'] + [col for col in df_c.columns if col not in ['DATE_ONLY', 'HOUR_ONLY']]
df_c = df_c[cols]

df_c = df_c.drop(['DATE','HOUR'],axis=1)

In [71]:
# extract date & hour
df_c_tvl['DATE'] = df_c_tvl['DATE'].astype(str).str.replace('\ufeff', '').str.strip()
df_c_tvl['HOUR'] = df_c_tvl['HOUR'].astype(str).str.replace('\ufeff', '').str.strip()

df_c_tvl['DATE'] = pd.to_datetime(df_c_tvl['DATE'], errors='coerce')
df_c_tvl['HOUR'] = pd.to_datetime(df_c_tvl['HOUR'], errors='coerce')

df_c_tvl['DATE_ONLY'] = df_c_tvl['DATE'].dt.date
df_c_tvl['HOUR_ONLY'] = df_c_tvl['HOUR'].dt.hour

cols = ['DATE_ONLY', 'HOUR_ONLY'] + [col for col in df_c_tvl.columns if col not in ['DATE_ONLY', 'HOUR_ONLY']]
df_c_tvl = df_c_tvl[cols]

df_c_tvl = df_c_tvl.drop(['DATE','HOUR'],axis=1)

In [73]:
df_c_tvl.head(1)

,DATE_ONLY,HOUR_ONLY,POOL,TVL
0,2025-04-06,19.0,Curve.fi Factory Plain Pool: crvUSD/USDC,NaN


In [74]:
df_c.head(1)

,DATE_ONLY,HOUR_ONLY,POOL,SWAP_COUNT,TOTAL_VOLUME_USD,MIN_DEPTH_USD,AVG_SLIPPAGE,MAX_SLIPPAGE,MAX_PRICE,MIN_PRICE,AVG_PRICE
0,2025-04-05,2.0,Curve.fi Factory Plain Pool: crvUSD/USDT,1.0,3299.25,3299.25,0.0091,0.0091,0.999909,0.999909,0.999909


In [80]:
df_c_tvl[df_c_tvl['TVL'].isnull()]['POOL'].value_counts()

Curve.fi Factory Plain Pool: crvUSD/USDT    4053
Curve.fi Factory Plain Pool: crvUSD/USDC    3860
Curve.fi Pool: DAI-USDC-USDT-sUSD            681
aDAI-aUSDT-DAI-USDC-aUSDC-USDT                19
Curve.fi Pool: cDAI-cUSDC-USDT                13
DAI-3Crv-USDC-USDT-mUSD                        6
USDK-DAI-3Crv-USDC-USDT                        5
cyUSDT-DAI-cyUSDC-cyDAI-USDC-USDT              4
LINKUSD-DAI-3Crv-USDC-USDT                     1
Name: POOL, dtype: int64

In [77]:
df_c_tvl['POOL'].value_counts()

Curve.fi Factory Plain Pool: crvUSD/USDT    4053
Curve.fi Factory Plain Pool: crvUSD/USDC    3860
DAI-USDC-USDT                                930
Curve.fi Pool: DAI-USDC-USDT-sUSD            681
FRAX-USDC                                    606
RAI-DAI-3Crv-USDC-USDT                       113
GUSD-DAI-3Crv-USDC-USDT                       37
aDAI-aUSDT-DAI-USDC-aUSDC-USDT                19
Curve.fi Pool: cDAI-cUSDC-USDT                13
USDC-USDT                                     12
DAI-3Crv-USDC-USDT-mUSD                        6
USDK-DAI-3Crv-USDC-USDT                        5
cyUSDT-DAI-cyUSDC-cyDAI-USDC-USDT              4
RSV-DAI-3Crv-USDC-USDT                         4
USDP-DAI-3Crv-USDC-USDT                        1
LINKUSD-DAI-3Crv-USDC-USDT                     1
USDN-DAI-3Crv-USDC-USDT                        1
DUSD-DAI-3Crv-USDC-USDT                        1
Name: POOL, dtype: int64

In [63]:
df_c_tvl['DATE_ONLY'] = df_c_tvl['DATE_ONLY'].astype(str)
df_c_tvl[(df_c_tvl['POOL_NAME']=="Curve.fi Pool: DAI-USDC-USDT-sUSD") & (df_c_tvl['DATE_ONLY']=='2025-06-01') &
        (df_c_tvl['HOUR_ONLY']==12.0)]

,DATE_ONLY,HOUR_ONLY,POOL_NAME,AVG_TVL_USD,PRICE_USD,SLIPPAGE_PERC
29182,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1949.36,1.0,0.0665
47536,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,NaN,1.0,0.0289
47583,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,NaN,1.0,0.0026


In [62]:
df_c['DATE_ONLY'] = df_c['DATE_ONLY'].astype(str)
df_c[(df_c['POOL_NAME']=="Curve.fi Pool: DAI-USDC-USDT-sUSD") & (df_c['DATE_ONLY']=='2025-06-01') &
    (df_c['HOUR_ONLY']==12.0)]

,DATE_ONLY,HOUR_ONLY,POOL_NAME,SWAP_COUNT,TOTAL_VOLUME_USD,MIN_DEPTH_USD,AVG_SLIPPAGE,MAX_SLIPPAGE,VOLATILITY_PERC
28,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.171377
318,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.171377
392,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.845399
436,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.845399
686,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.171377
...,...,...,...,...,...,...,...,...,...
98453,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.190249
98767,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.190249
98786,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.079199
99258,2025-06-01,12.0,Curve.fi Pool: DAI-USDC-USDT-sUSD,1.0,2009.87,2009.87,0.028862,0.028862,0.312569


In [87]:
import requests
import pandas as pd

r = requests.get("https://stablecoins.llama.fi/stablecoins?includePrices=true")
data = r.json()
https://api.llama.fi/protocol/curve

# https://api.llama.fi/v2/historicalChainTvl/Ethereum
df = pd.DataFrame(data)
#df['date'] = pd.to_datetime(df['date'], unit='s')


ValueError: All arrays must be of the same length

In [84]:
df

,date,tvl
0,2018-04-27,404078
1,2018-04-28,400585
2,2018-04-29,429214
3,2018-04-30,421651
4,2018-05-01,436195
...,...,...
2592,2025-06-01,60932494622
2593,2025-06-02,61467453824
2594,2025-06-03,62076825001
2595,2025-06-04,62063644375


In [104]:
import requests
import pandas as pd

url = "https://stablecoins.llama.fi/stablecoins?includePrices=true"
response = requests.get(url)
data = response.json()

# دیتا داخل کلید 'peggedAssets' هست
df = pd.json_normalize(data['peggedAssets'])

In [105]:
df

,id,name,symbol,gecko_id,pegType,priceSource,pegMechanism,chains,price,circulating.peggedUSD,...,chainCirculating.Swellchain.circulatingPrevDay.peggedUSD,chainCirculating.Swellchain.circulatingPrevWeek.peggedUSD,chainCirculating.Swellchain.circulatingPrevMonth,chainCirculating.OP Mainnet.circulatingPrevMonth,chainCirculating.Scroll.circulatingPrevMonth,chainCirculating.Avalanche.circulatingPrevMonth,chainCirculating.Arbitrum.circulatingPrevMonth,chainCirculating.Base.circulatingPrevMonth,chainCirculating.Stellar.circulatingPrevMonth,chainCirculating.Polygon.circulatingPrevMonth
0,1,Tether,USDT,tether,peggedUSD,defillama,fiat-backed,"[Tron, Ethereum, BSC, Solana, Polygon, Arbitru...",1,1.554050e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,USD Coin,USDC,usd-coin,peggedUSD,defillama,fiat-backed,"[Ethereum, Solana, Base, Hyperliquid L1, Arbit...",0.999763,6.065978e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,TerraClassicUSD,USTC,terrausd,peggedUSD,coingecko,algorithmic,"[OP Mainnet, Metis, Terra Classic, DFK, Moonbe...",0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Binance USD,BUSD,binance-usd,peggedUSD,defillama,fiat-backed,"[Ethereum, IoTeX, Avalanche, Solana, Osmosis, ...",0.989165,5.779268e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Dai,DAI,dai,peggedUSD,defillama,crypto-backed,"[Ethereum, Polygon, Gnosis, BSC, Fantom, Arbit...",0.999739,4.347228e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,267,Mead,MEAD,mead-2,peggedUSD,defillama,crypto-backed,[Berachain],None,7.248066e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,268,YU,YU,yala-stablecoin,peggedUSD,defillama,crypto-backed,"[Ethereum, Solana]",1,2.265238e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,269,Liquity BOLD,BOLD,liquity-bold,peggedUSD,defillama,crypto-backed,"[Ethereum, Base, OP Mainnet, Arbitrum, Avalanc...",0.984786,3.322391e+07,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
257,270,Franklin Onchain U.S. Government Money Fund,BENJI,franklin-onchain-u-s-government-money-fund,peggedUSD,defillama,fiat-backed,"[Stellar, Arbitrum, Base, Avalanche, Polygon]",None,6.856562e+08,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [90]:
df['name'].value_counts()

Tether                   1
Dyad                     1
Fathom Dollar            1
UNO                      1
Web 3 Dollar             1
                        ..
Digital Standard Unit    1
Monerium EUR emoney      1
Offshift anonUSD         1
NXUSD                    1
Avant USD                1
Name: name, Length: 259, dtype: int64

In [ ]:
df_current_pools

In [106]:
import requests
import pandas as pd

# مرحله 1: گرفتن تمام استخرها
url = "https://yields.llama.fi/pools"
response = requests.get(url)
all_pools = response.json()['data']

# فیلتر استخرهایی که روی Ethereum هستند و پروژه Curve دارند
curve_ethereum_pools = [
    pool for pool in all_pools
    if pool['project'].lower() == 'curve' and pool['chain'].lower() == 'ethereum'
]

# تبدیل به DataFrame برای ذخیره اطلاعات اولیه
df_curve_eth = pd.DataFrame([{
    'pool_id': pool['pool'],
    'pool_name': pool.get('symbol', ''),
    'chain': pool['chain'],
    'project': pool['project']
} for pool in curve_ethereum_pools])

In [107]:
df_curve_eth

""


In [4]:
import requests
import pandas as pd

# مرحله 1: گرفتن تمام استخرها
url = "https://yields.llama.fi/pools"
response = requests.get(url)
all_pools = response.json()['data']

# فیلتر استخرهایی که روی Ethereum هستند و پروژه Curve دارند
curve_ethereum_pools = [
    pool for pool in all_pools
    if pool['project'] == 'curve-dex' and pool['chain'] == 'Ethereum'
]

# تبدیل به DataFrame برای ذخیره اطلاعات اولیه
df_curve_eth = pd.DataFrame([{
    'chain': pool['chain'],
    'project': pool['project'],
    'pool_name': pool.get('symbol', ''),
    'pool_id': pool['pool'],
    'stablecoin' : pool['stablecoin'],
    'tvlUsd' : pool['tvlUsd']

} for pool in curve_ethereum_pools])

In [6]:
def format_number(n):
    if n >= 1e9:
        return f"{n/1e9:.2f}B"
    elif n >= 1e6:
        return f"{n/1e6:.2f}M"
    elif n >= 1e3:
        return f"{n/1e3:.2f}K"
    else:
        return str(n)

df_curve_eth['tvlUsd_pretty'] = df_curve_eth['tvlUsd'].apply(format_number)

In [7]:
df_curve_eth

,chain,project,pool_name,pool_id,stablecoin,tvlUsd,tvlUsd_pretty
0,Ethereum,curve-dex,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,True,182088963,182.09M
1,Ethereum,curve-dex,ETH-STETH,57d30b9c-fc66-4ac2-b666-69ad5f410cce,False,120546571,120.55M
2,Ethereum,curve-dex,REUSD-SCRVUSD,5c4940c7-c193-440d-b95e-9148d017e12c,False,98786998,98.79M
3,Ethereum,curve-dex,USD0-USD0++,bc58a174-05f8-4e41-8f77-be9c13a614bf,True,79461688,79.46M
4,Ethereum,curve-dex,FRAX-USDE,12ca9565-0369-404e-b209-631305e4012a,True,67947399,67.95M
...,...,...,...,...,...,...,...
398,Ethereum,curve-dex,FETH-FRAXBP,48fbcc9f-a12d-44ad-994a-caa81244bc17,False,10725,10.72K
399,Ethereum,curve-dex,REUSD-USDC,3c7d73df-4886-4184-9d40-d60c09f4d289,True,10064,10.06K
400,Ethereum,curve-dex,WA7A5-CRVUSD,258daf2d-2ccf-4f86-9c63-14a6a5dba998,False,10054,10.05K
401,Ethereum,curve-dex,USDB-USDC,5f996dd0-f86e-44ea-83c2-1fb0bdd421b4,True,10052,10.05K


# TVL per pool

In [29]:
df_c = df_curve_eth[df_curve_eth['stablecoin']==True]
df_c1 = df_c.groupby(['pool_name','pool_id'])['tvlUsd'].mean().reset_index().sort_values(by='tvlUsd', ascending=False )
df_c1['tvlUsd_pretty'] = df_c1['tvlUsd'].apply(format_number)
df_c1['share'] = round((df_c1['tvlUsd'] / df_c1['tvlUsd'].sum()) * 100)

In [31]:
df_c1.head(20)

,pool_name,pool_id,tvlUsd,tvlUsd_pretty,share
14,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,182088963.0,182.09M,20.0
79,USD0-USD0++,bc58a174-05f8-4e41-8f77-be9c13a614bf,79461688.0,79.46M,9.0
42,FRAX-USDE,12ca9565-0369-404e-b209-631305e4012a,67947399.0,67.95M,7.0
86,USDC-RLUSD,e91e23af-9099-45d9-8ba5-ea5b4638e453,54562565.0,54.56M,6.0
78,SUSDS-USDT,6f74baf2-48f3-4e4a-ad67-5c056f37e4fa,50646630.0,50.65M,6.0
13,CUSDO-USDC,08d1f13b-5d65-42c5-863d-fcb447dcea75,36027432.0,36.03M,4.0
25,DOLA-USR,d78803f4-437f-4ce5-b199-1bae142a185d,31192778.0,31.19M,3.0
70,SDAI-SUSDE,d22deafb-dbb1-417c-b555-cc8c9f4b955a,22188816.0,22.19M,2.0
97,USDL-USDC,b92d0d7b-6113-4981-9fa7-fad4d20cad6b,20936300.0,20.94M,2.0
80,USD0-USDC,23a68630-c706-4ed3-bb31-92868a15050b,20924338.0,20.92M,2.0


In [32]:
df_c1.shape

(119, 5)

In [42]:
import requests
import pandas as pd
from datetime import datetime

pool_info = {
    "25171c4c-1877-449a-9f88-45a9f153ee31": "DAI-USDC-USDT",
    "bc58a174-05f8-4e41-8f77-be9c13a614bf": "USD0-USD0++",
    "12ca9565-0369-404e-b209-631305e4012a": "FRAX-USDE",
    "e91e23af-9099-45d9-8ba5-ea5b4638e453": "USDC-RLUSD",
    "6f74baf2-48f3-4e4a-ad67-5c056f37e4fa": "SUSDS-USDT",
    "08d1f13b-5d65-42c5-863d-fcb447dcea75": "CUSDO-USDC",
    "d78803f4-437f-4ce5-b199-1bae142a185d": "DOLA-USR",
    "d22deafb-dbb1-417c-b555-cc8c9f4b955a": "SDAI-SUSDE",
    "b92d0d7b-6113-4981-9fa7-fad4d20cad6b": "USDL-USDC",
    "23a68630-c706-4ed3-bb31-92868a15050b": "USD0-USDC",
    "fa91389d-ab0f-43e5-8921-9c9a462d97d2": "USDC-USDTB",
    "755529b5-fcf4-4ef0-a7c7-e4f49376706f": "BOLD-USDC",
    "8a3a7995-73b3-4e25-8b5c-82a8841bf720": "DOLA-SUSDE",
    "bcc046f9-d980-40d9-80b2-5d5ae5931131": "USDC-FXUSD",
    "b36c0e1b-792f-4fee-8160-a0002c23e1d1": "DOLA-SUSDS",
    "3388eabd-fc53-49bf-b358-d1a6dc3c6112": "USDC-USDF",
    "14681aee-05c9-4733-acd0-7b2c84616209": "PYUSD-USDC",
    "985b3a44-495f-4250-bde3-7b2299036e3e": "DAI-USDC-USDT-SUSD",
    "f0eead86-5acd-41a9-93c9-b01766d120e6": "FRAX-SDAI",
    "3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1": "FRAX-USDC"
}

start_date = datetime(2025, 5, 1)
end_date = datetime(2025, 6, 7)

results = []

for pool_id, pool_name in pool_info.items():
    url = f"https://yields.llama.fi/chart/{pool_id}"
    r = requests.get(url)
    if r.status_code != 200:
        print(f"❌ Failed for pool {pool_id}")
        continue

    data = r.json().get("data", [])

    for entry in data:
        ts = datetime.fromisoformat(entry["timestamp"].replace("Z", "")).replace(tzinfo=None)
        if start_date <= ts <= end_date:
            results.append({
                "pool_name": pool_name,
                "pool_id": pool_id,
                "timestamp": ts.date(),  # فقط تاریخ
                "tvlUsd": entry.get("tvlUsd", None)
            })

# 📊 تبدیل به DataFrame
df_result = pd.DataFrame(results)


In [43]:
df_result

,pool_name,pool_id,timestamp,tvlUsd
0,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-01,175508228
1,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-02,175492317
2,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-03,175114027
3,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-04,175365655
4,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-05,175832607
...,...,...,...,...
717,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-06-02,11409570
718,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-06-03,11136791
719,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-06-04,11591205
720,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-06-05,11267097


In [44]:
df_result['pool_name'].value_counts()

DAI-USDC-USDT         37
USD0-USD0++           37
FRAX-SDAI             37
DAI-USDC-USDT-SUSD    37
PYUSD-USDC            37
USDC-USDF             37
DOLA-SUSDS            37
USDC-FXUSD            37
DOLA-SUSDE            37
USDC-USDTB            37
USD0-USDC             37
USDL-USDC             37
SDAI-SUSDE            37
DOLA-USR              37
CUSDO-USDC            37
SUSDS-USDT            37
USDC-RLUSD            37
FRAX-USDE             37
FRAX-USDC             37
BOLD-USDC             19
Name: pool_name, dtype: int64

In [45]:
df_result.to_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/onchain-analytics/01_Market_Making/df_tvl.csv')